In [1]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador,Bawm
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
from pretty_html_table import build_table
import numpy as np

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
crm = Crm()
bds = BDS()

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = bds.banco.hoje()

#Posicao_Fundos hoje
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=False)
posicao_fundos['GuidProduto'] = posicao_fundos['GuidProduto'].str.lower()

#PL dos fundos
pl = posicao.fundos_pl_estimado()[['NomeContaCRM','PL']]

#Cadastro dos fundos
cadastro = posicao.po_cadastro()

#Informações dos gestores
fundos = crm.fundos_crm()
fundos['Productid']= fundos['Productid'].str.lower()
fundos = fundos[['Productid','new_gestoridname','CNPJ_Gestor']]

#Retorna a posição explodida por sc

portfolio_jbb = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/06 - Relatório/Tableau/posdia_Explosao.xlsx')

#retorna os tickers de RV para verificar os limites por ação dentro dos portfolios
rv = bds.consulta_ativos_RV()
rv['productid'] = rv['productid'].str.lower()
rv = rv[['productid','Nome']]

#retira tudo o que foi FII do RV
tipo_produtos = posicao_fundos[['GuidProduto','TipoProduto']].drop_duplicates()
tipo_produtos = posicao_fundos[posicao_fundos['TipoProduto']!='FII']
rv = pd.merge(left=tipo_produtos,right=rv,left_on='GuidProduto',right_on='productid',how='left').dropna(subset=['Nome'])

#Limites por ação dentro dos portfolios
limite_rv = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/2064_Brasil/Limites - Políticas 2064.xlsx',sheet_name='Limite RV')
min = limite_rv[limite_rv['%']!=0].min()[1]

In [3]:
#Verifica os limites por ação dentro do portfolio, de acordo com o perfil do cliente

portfolio_rv = portfolio_jbb[['SC','Perfil','PL','GuidProduto','FinanceiroFinal']]
portfolio_rv['GuidProduto'] = portfolio_rv['GuidProduto'].str.lower()
portfolio_rv = pd.merge(left=rv, right=portfolio_rv,left_on='productid',right_on='GuidProduto',how='left')
portfolio_rv = portfolio_rv.dropna(subset=['GuidProduto'])

#Faz a consolidação por ação
portfolio_rv = portfolio_rv.groupby(['SC','Nome','Perfil','PL']).sum().reset_index()
portfolio_rv['%PL'] = portfolio_rv['FinanceiroFinal'] / portfolio_rv['PL']

#Verifica o tipo de portfolio e o limite.
portfolio_rv = pd.merge(left=limite_rv, right=portfolio_rv,left_on='Limite ação direta por portfolio (nível global)',right_on='Perfil',how='right')

# Preenchendo o que está nulo com o minimo da tabela
portfolio_rv = portfolio_rv[['SC','Nome','Perfil','%PL','%']].fillna(min)

#Verificando clientes desenquadrados
portfolio_rv['Desenquadrado']= portfolio_rv['%PL'] > portfolio_rv['%']
portfolio_rv = portfolio_rv.sort_values(by=['%PL'],ascending=False)

In [4]:
## Posicao JBB pno AUM da gestora

#Posicao AUM JBB
posicao_gestor = posicao_fundos[['GuidProduto','FinanceiroFinal','TipoProduto']]

#Linkando com o cadastro dos gestores (CNPJ da gestora)
posicao_gestor = pd.merge(left=fundos, right=posicao_gestor,left_on='Productid',right_on='GuidProduto',how='inner')

#Somando o total que temos por gestora (AUM)
posicao_gestor = posicao_gestor.groupby(['new_gestoridname','CNPJ_Gestor']).sum().reset_index()[['new_gestoridname','CNPJ_Gestor','FinanceiroFinal']].rename(columns={'new_gestoridname':'Nome_Gestor'}).sort_values(by=['FinanceiroFinal'],ascending=False)
posicao_gestor.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/2064_Brasil/posicao_gestor.xlsx')

In [5]:
## Posicao total do gestor dentro do fundo exclusivo / coletivo

#Linkando com o cadastro dos gestores (CNPJ da gestora)
liquidez = ['CAIXA','COMP','Ajustes','Caixa','AJUSTES']
fundos_zeragem = ['5aa8e514-0fc2-dd11-9886-000f1f6a9c1c','838559d4-3f1f-ee11-8d7a-005056b17af5','fee2528e-541d-ec11-a569-005056912b96','8ba7e514-0fc2-dd11-9886-000f1f6a9c1c','b4403d0b-185f-eb11-9d30-005056912b96','f4a7e952-713a-e311-87ef-000c29cb7e20','c9023af7-6f01-e111-a443-d8d385b9752e','d9e2871d-f542-ef11-b0a5-005056b17af5','2c44617a-e483-e911-8f6a-005056912b96']


posicao_fundo_gestor = posicao_fundos.loc[~posicao_fundos['TipoProduto'].isin(liquidez)]
posicao_fundo_gestor['GuidProduto'] = posicao_fundo_gestor['GuidProduto'].str.lower()
posicao_fundo_gestor = posicao_fundo_gestor.loc[~posicao_fundo_gestor['GuidProduto'].isin(fundos_zeragem)]
posicao_fundo_gestor.to_excel('verificar.xlsx')

#Retirar todos os fundos que possuem menos de 2 produtos (retirar os feeders)
posicao_ativos = posicao_fundo_gestor.groupby(['NomeContaCRM']).count()[['Fonte']].reset_index()
posicao_ativos.to_excel('verificarv2.xlsx')
posicao_ativos = posicao_ativos[posicao_ativos['Fonte']>3]
posicao_ativos = posicao_ativos[~posicao_ativos['NomeContaCRM'].str.contains('Monitorado')]
posicao_ativos = posicao_ativos[~posicao_ativos['NomeContaCRM'].str.contains('VDSV14')]

posicao_fundo_gestor = pd.merge(left=fundos, right=posicao_fundo_gestor,left_on='Productid',right_on='GuidProduto',how='inner')

#Somando o total que temos por gestora (AUM)
posicao_fundo_gestor = posicao_fundo_gestor.groupby(['GuidContaCRM','NomeContaCRM','new_gestoridname','CNPJ_Gestor']).sum().reset_index()[['GuidContaCRM','NomeContaCRM','new_gestoridname','CNPJ_Gestor','FinanceiroFinal']].rename(columns={'new_gestoridname':'Nome_Gestor'}).sort_values(by=['FinanceiroFinal'],ascending=False)

#Tipo do fundo (exclusivo, previdencia, fundos coletivos)
tipo_fundo = cadastro[['NomeContaCRM','TipoCobranca','TipoCliente']]

#Encontrar o PL do fundo e ver o % que representa o gestor no PL
posicao_fundo_gestor = pd.merge(left = posicao_fundo_gestor, right=pl, on='NomeContaCRM',how='left')
posicao_fundo_gestor['%PL'] = posicao_fundo_gestor['FinanceiroFinal'] / posicao_fundo_gestor['PL']
posicao_fundo_gestor = pd.merge(left = posicao_fundo_gestor, right=tipo_fundo, on='NomeContaCRM',how='left')

#Retira tudo o que for possivelmente feeder
posicao_fundo_gestor = pd.merge(left = posicao_fundo_gestor, right=posicao_ativos, on='NomeContaCRM',how='right')
posicao_fundo_gestor = posicao_fundo_gestor.sort_values(by=['%PL'],ascending=False).drop(columns=['Fonte'])


#Retirar NA e valores que não fazem sentido
posicao_fundo_gestor = posicao_fundo_gestor.dropna(subset=['%PL'])
posicao_fundo_gestor = posicao_fundo_gestor[posicao_fundo_gestor['%PL']!=np.inf]

#Posição dos fundos por gestor ex JBB
posicao_fundo_gestor = posicao_fundo_gestor[posicao_fundo_gestor['Nome_Gestor']!='Julius Baer Family Office Brasil Gestão de Patrimônio Ltda']
posicao_fundo_gestor = posicao_fundo_gestor[['NomeContaCRM','Nome_Gestor','CNPJ_Gestor','FinanceiroFinal','PL','%PL','TipoCobranca','TipoCliente']].reset_index(drop=True)

posicao_fundo_gestor.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/2064_Brasil/posicao_gestor_pl_fundo.xlsx')

In [9]:
#Posicao do gestor por sc
posicao_gestor_sc = portfolio_jbb[['SC','Perfil','GuidProduto','FinanceiroFinal','PLFuturo',]]
posicao_gestor_sc = pd.merge(left=fundos, right=posicao_gestor_sc,left_on='Productid',right_on='GuidProduto',how='inner')

#Somando o total que temos por gestora (AUM)
posicao_gestor_sc = posicao_gestor_sc.groupby(['SC','Perfil','new_gestoridname','PLFuturo']).sum().reset_index()

#Tirando do Calculo o que é JBB
posicao_gestor_sc = posicao_gestor_sc[posicao_gestor_sc['new_gestoridname']!='Julius Baer Family Office Brasil Gestão de Patrimônio Ltda']

#Retornar o quanto da posição do gestor representa do PL da SC
posicao_gestor_sc['%PL'] = posicao_gestor_sc['FinanceiroFinal'] / posicao_gestor_sc['PLFuturo']

# Ordenar o % do maior para o menor e retirar o que estiver com NA
posicao_gestor_sc = posicao_gestor_sc.sort_values(by=['%PL'],ascending=False).dropna(subset=['%PL']).reset_index(drop=True)

#verificar se tem algum cliente com mais de 6% de posição por gestor
PL = posicao_gestor_sc['%PL']
PL = posicao_gestor_sc['%PL']
desenquadrados = ['Desenquadrado' if x >0.06 else 'ok' for x in PL]
posicao_gestor_sc['Status'] = desenquadrados


In [10]:
#Consolidação das informações e salvando apenas um excel

#Gerando excel para e-mail:
with pd.ExcelWriter('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/2064_Brasil/Controle_2064.xlsx') as writer:  
    portfolio_rv.to_excel(writer, sheet_name='Posicao_RV_SC')
    posicao_gestor.to_excel(writer, sheet_name='Posicao_JBB_Gestora')
    posicao_fundo_gestor.to_excel(writer, sheet_name='Posicao_Fundo_Gestora')
    posicao_gestor_sc.to_excel(writer, sheet_name='Posicao_Gestora_SC')